In [1]:
import numpy as np
import pandas as pd

import refinitiv.dataplatform.eikon as ek
import option_ric_tools as ort

ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

Let's start by defining what data we will be fetching. We will need to define the following:
* The ISIN for the underlying security as a string
* The maturity date for any options as a string
* The strike prices for which to fetch options prices as a list of integers
* The kinds of options (call/put) to fetch prices for as a list of strings (allowed values are 'C' and 'P')

An example for IBM is provided below. If you are unsure what strikes to fetch you might want to check the price of the underlying 90 days prior to your chosen maturity. Any configurations for which no prices are available will be skipped.

In [2]:
isin = 'US0378331005'
maturity = '2024-01-19'
strikes = [170, 175, 180, 185, 190]
kinds = ['C', 'P']

The code below will then try to fetch pricing for what we defined. Using the Eikon API requires quite a lot of preprocessing, which is taken care of in the file 'option_ric_tools.py' The price series for each option will be added as a column.

In [3]:
price_dfs = []
for kind in kinds:
    assert kind in ['C', 'P']
    for strike in strikes:
        kind_str = 'call' if kind == 'C' else 'P'
        print(f'\nFetching prices for {kind_str} option with strike price {strike}')
        rics, prices_list = ort.get_optionRic(isin, maturity, strike, kind)
        try:
            price_df = prices_list[0]
            price_df = price_df.drop(['BID', 'ASK'], axis=1)
            price_df.columns = [f'{kind}{strike}']
            price_dfs.append(price_df)
        except IndexError:
            print('Skipping empty combination')

price_df = pd.concat(price_dfs, axis=1)


Fetching prices for call option with strike price 170
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 175
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 180
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 185
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 190
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 170
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 175
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 180
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 185
Option RIC for OPQ exchange is successfully constructed

Fetching pri

In [4]:
price_df.tail(10)

,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190
2023-11-03,12,8.57,5.75,3.47,1.94,3.5,5.06,7.34,10.25,14.1
2023-11-06,13.73,10.1,6.91,4.32,2.47,2.77,4.05,5.9,8.5,12
2023-11-07,15.8,11.9,8.45,5.5,3.3,2.15,3.25,4.77,6.9,10
2023-11-08,16.56,12.56,9,5.93,3.5,1.9,2.88,4.3,6.25,9
2023-11-09,16.1,12.16,8.35,5.61,3.33,2.06,3.06,4.53,6.75,9.8
2023-11-10,19.65,15.35,11.37,7.88,5,1.31,1.98,3.01,4.55,6.7
2023-11-13,18.18,13.86,10.07,6.8,4.16,1.46,2.24,3.46,5.2,7.6
2023-11-14,20.37,15.84,11.82,8.16,5.27,0.97,1.56,2.47,3.93,5.93
2023-11-15,20.75,16.25,12.3,8.55,5.5,0.92,1.42,2.29,3.6,5.57
2023-11-16,22.22,17.8,13.56,9.8,6.4,0.86,1.29,2.02,3.17,4.9


Let's now fetch the price series for the underlying for the same time period. Eikon uses a special identifier called RIC. The code below find the corresponding RIC for the ISIN we defined. It also find the earliest and latest dates in our options price series.

In [5]:
ric = ek.get_symbology(symbol=isin, from_symbol_type='ISIN', to_symbol_type='RIC').iloc[0,0]
sdate = str(price_df.index.min().date())
edate = str(price_df.index.max().date())

We can now fetch the price series for the underlying and add it as a column to the dataframe.

In [6]:
stock = ek.get_timeseries(
    rics=[ric],
    fields=['CLOSE'],
    start_date=sdate,
    end_date=edate,
    interval='daily'
)

stock = stock.reset_index()
stock.columns.name = None
stock.index = stock['Date']
stock.index.name = None
stock = stock.drop('Date', axis=1)
stock.columns = ['Underlying']

price_df = stock.join(price_df)

In [7]:
price_df.head()

,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190
2023-08-21,175.84,16.4,13.27,10.3,7.85,5.91,6.81,8.65,10.85,13.35,16.5
2023-08-22,177.23,17.5,14.1,11.1,8.51,6.34,6.26,8.1,10.1,12.75,15.5
2023-08-23,181.12,20.2,16.65,13.45,10.55,8.08,5.25,6.71,8.5,10.7,13.33
2023-08-24,176.38,16.97,13.75,10.8,8.35,6.25,6.85,8.62,10.75,13.35,16.6
2023-08-25,178.61,18.5,15.15,11.85,9.19,6.9,5.9,7.5,9.5,11.9,14.81


As a final step we save the constructed dataframe for further analysis. The default code saves the dataframe in feather format, but if you can issues you can uncomment the lines further down to export to csv.

In [8]:
price_df.index.name = 'Date'
price_df = price_df.reset_index()
price_df.to_feather('fe2.feather')
price_df.to_csv('fe2.csv')